In [3]:
from typing import Dict, List
from bespokelabs import curator
from pydantic import BaseModel, Field
import os

API_ENDPOINTS = {
    "bailian": {
        "url": "https://dashscope.aliyuncs.com/compatible-mode/v1",
        "key": "sk-987eabba7cf9404d9bde119440ce561a",
        "model": "qwen3-max"
    },
    "internal/R1": {
        "url": "http://10.200.48.200:30600/v1",
        "key": "sk-H8RJWNGwqEmIfk2ushYsqWWCbrjdReoEOwxqB6GAS5kZkyk9",
        "model": "deepseek-ai/DeepSeek-R1"
    },
    "internal/qwen-plus": {
        "url": "http://10.200.48.200:30600/v1",
        "key": "sk-H8RJWNGwqEmIfk2ushYsqWWCbrjdReoEOwxqB6GAS5kZkyk9",
        "model": "qwen-plus-2025-07-28"
    },
    "internal/v3": {
        "url": "http://10.200.48.200:30600/v1",
        "key": "sk-H8RJWNGwqEmIfk2ushYsqWWCbrjdReoEOwxqB6GAS5kZkyk9",
        "model": "deepseek-ai/DeepSeek-V3"
    },
    "Horayai": {
        "url": "https://sr-endpoint.horay.ai/openai/v1",
        "key": "sk-hzaoqnkkkubhulblxprriyhldguxphmydljwguivffwjvbwx",
        "model": "g5.1"
    }
}

api_conf = API_ENDPOINTS['internal/R1']

os.environ["OPENAI_BASE_URL"] = api_conf['url']  # base_url
os.environ["OPENAI_API_KEY"] = api_conf['key']  # api_key
os.environ["OPENAI_MODEL_NAME"] = api_conf['model']  # model_name

class Topics(BaseModel):
    topics_list: List[str] = Field(description="A list of topics.")

class TopicGenerator(curator.LLM):
  response_format = Topics

  def prompt(self, subject):
    return f"Return 3 topics related to {subject}"

  def parse(self, input: str, response: Topics):
    return [{"topic": t} for t in response.topics_list]


class Poem(BaseModel):
    title: str = Field(description="The title of the poem.")
    poem: str = Field(description="The content of the poem.")

class Poet(curator.LLM):
    response_format = Poem

    def prompt(self, input: Dict) -> str:
        return f"Write two poems about {input['topic']}."

    def parse(self, input: Dict, response: Poem) -> Dict:
        return [{"title": response.title, "poem": response.poem}]

topic_generator = TopicGenerator(model_name=api_conf['model'])
poet = Poet(model_name=api_conf['model'])
# Start generation
topics = topic_generator("Mathematics")
poems = poet(topics)


Provider List: https://docs.litellm.ai/docs/providers



BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=deepseek-ai/deepseek-r1
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers